In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataretrieval as nwis
from tqdm import tqdm
import datetime as dt

In [ ]:
NPSites = nwis.get_info(huc = '17',parameterCd = '00060', startDt = '1950-01-01',endDt = '2020-12-31',siteType='ST',seriesCatalogOutput = True)[0]
NPSites = NPSites[(NPSites["count_nu"]>365*2)&(NPSites["agency_cd"]=="USGS")] # More than 2 years of data and operated by USGS
siteNum = NPSites["site_no"].unique() ## Get unique site numbers from the above dataset

In [ ]:
startDate = "1950-01-01"
endDate = "2020-12-31"

dvstats = pd.DataFrame(columns= ['numZero','numOne','startDate','endDate'])
ivstats = pd.DataFrame(columns= ['numZero','numOne','startDate','endDate'])

for i in tqdm(siteNum,leave=True):
    print(i)
    df = nwis.get_dv(sites=str(i),
           start = startDate,
           end = endDate)[0]
    
    
    if '00060_Mean' in df.columns and len(df[df['00060_Mean'] < 1]) >= 7:
        dvstats.loc[str(i)] = (len(df[df["00060_Mean"]==0]),len(df[df["00060_Mean"]<=1]),df.index[1],df.index[-1])
        df.to_csv("/Users/adamprice/Desktop/NWIS_streamflow/daily/"+str(i)+"_daily.csv")
        del(df)
        
#         iv = nwis.get_iv(sites=str(i),
#            start = startDate,
#            end = endDate)[0]
        
#         if len(iv) == 0:
#             continue
#         else:
#             try:
#                 ivstats.loc[str(i)] = (len(iv[iv["00060"]==0]),len(iv[iv["00060"]<=1]),iv.index[1],iv.index[-1])
#                 iv.to_csv("/Users/adamprice/Desktop/NWIS_streamflow/unitVal/"+str(i)+"_unitVal.csv")
#                 del(iv)
#             except:
# #                 ivstats.loc[str(i)] = ("n/a","n/a","n/a","n/a")
#                 continue
        
    else:
        continue
        
dvstats['startDate'] = dvstats['startDate'].dt.tz_localize(None)
dvstats['endDate'] = dvstats['endDate'].dt.tz_localize(None)
dvstats.to_csv("../data/pnwNWall_DailyStats.csv")
# ivstats.to_csv("../data/pnwNWall_UnitStats.csv")
        